In [1]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11,8)})
sns.set(font_scale=1.2)
import numpy as np

In [2]:
df_2022 = pd.read_csv("../original_data/ImmiRiskIPT2022_DATA_2022-12-12_1719.csv", na_values= ' ')
df_2021 = pd.read_csv(r"../original_data/Schneider Depression Clinic Database.csv", na_values= ' ')


C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (44,51,53,99,325,366,439,453,459,464,466,501,507,512,514,521,530,532,569,571,580,599,766,771,777,781,786,788,793,795,804,806,826,843,845,854,873,883,918,941,979,991,1002,1040,1045,1047,1049,1051,1053,1054,1080,1096,1119,1122,1126,1127,1131,1132,1133,1134,1136,1137,1138,1141,1142,1144,1161,1183,1186,1237,1239,1259,1261,1262,1263,1269,1270,1271,1272,1384,1387,1389,1405,1408,1411,1415,1432,1434,1435,1436,1442,1443,1444,1445,1499,1568,1569,1572,1573) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [51]:
def is_solved(col_name):
    if col_name.endswith('complete'):
        return True
    elif col_name.startswith('diagnosis'):
        return True
    elif col_name.startswith('complaint'):
        return True
    elif col_name.endswith('timestamp'):
        if 'opening' in col_name:
            return True
    
    elif col_name.startswith('group'):
        return True
    elif col_name.startswith('opening_psychothe'):
        return True
    elif col_name.startswith('opening_medicine'):
        return True
    elif col_name.startswith('with_who_'):
        return True
    
    elif col_name.startswith('chameleon_'):
        return True
    elif col_name.startswith('visit_date_stu'):
        return True
    elif col_name.startswith('treatment_end_2_stu'):
        return True
    
    return False

In [52]:
mult = [i for i in df_2022.columns if (i in df_2021.columns) and (not is_solved(i))]

In [ ]:
import re

def is_solved(col_name:str)-> bool:
    """
    This function takes in a column name as a parameter and checks if it meets any of the specified conditions.
    If the column name matches any of the conditions, it returns True. Otherwise, it returns False.
    
    Parameters:
        col_name (str): The column name to be checked
    
    Returns:
        bool: True if the column name meets any of the conditions, False otherwise
    """
    patterns = [r'.*complete$', r'^diagnosis.*', r'^complaint.*', r'.*timestamp$', r'^group.*', r'^opening_psychothe.*',
                r'^opening_medicine.*', r'^with_who_.*', r'^chameleon_.*', r'^visit_date_stu.*', r'^treatment_end_2_stu.*']
    for pattern in patterns:
        if re.match(pattern, col_name):
            return True
    return False

mult = [i for i in df_2022.columns if (i in df_2021.columns) and (not is_solved(i))]

In [53]:
y = df_2022.dropna(subset=['id'])


def get_first_id(record_id):
    ID = None
    dd = y[y.record_id == record_id]['id']
    if dd.empty:
        print(record_id)
    else:
        ID = dd.iloc[0]
    return ID



In [54]:
df_2022['id'] = df_2022['record_id'].apply(get_first_id)

85
550


In [55]:
combined = pd.concat([df_2022,df_2021])
combined = combined[combined.duplicated(subset=['id', 'redcap_event_name'], keep=False)]
combined = combined.sort_values(['id', 'redcap_event_name']).reset_index().drop('index', axis=1)

In [56]:
def find_diffs(s2):
    diff = []
    
    for i in mult:
        if not s2[i].isna().any():
            if s2.iloc[1][i] != s2.iloc[0][i]:
                diff.append(i)
    return diff

In [57]:
diff_dicts = []

In [58]:
for i in combined['id'].unique():
    for j in combined.redcap_event_name.unique():
        if not combined[(combined['id'] == i) & (combined.redcap_event_name == j)].empty:
            diff = find_diffs(combined[(combined['id'] == i) & (combined.redcap_event_name == j)])
            if len(diff):
                diff_dicts.append({'id': i, 'redcap_event_name':j, 'differences': diff})

In [59]:
diffs = pd.DataFrame(diff_dicts)

In [64]:
ids = 'A1707'
event_name = 'intake_arm_1'
col = 'ending_timestamp'

In [65]:
df_2021[(df_2021['id'] == ids) & (df_2021.redcap_event_name == event_name)][col]

1031    11/16/2021 5:01:11
Name: ending_timestamp, dtype: object

In [66]:
df_2022[(df_2022['id'] == ids) & (df_2022.redcap_event_name == event_name)][col]

670    2022-07-26 15:40:56
Name: ending_timestamp, dtype: object

In [63]:
diffs.iloc[:10]

,id,redcap_event_name,differences
0,A1707,intake_arm_1,"[age_child_pre, ending_timestamp]"
1,A5327,intake_arm_1,[record_id]
2,A5327,control_5weeks_arm_1,[record_id]
3,A5327,5th_session_arm_1,[record_id]
4,APPD9667,control_6month_arm_1,[treatment_end_stu]
5,D4440,intake_arm_1,[iat_done_no]
6,D6169,followup_3month_arm_1,[record_id]
7,D6169,intake_arm_1,[record_id]
8,D6169,control_5weeks_arm_1,[record_id]
9,D6169,5th_session_arm_1,[record_id]


In [34]:
ids = 'T4075'
event_name = 'control_5weeks_arm_1'
col = 'dass_10_f'

In [14]:
[i for i in df_2021 if i not in df_2022]

['ID_Merge',
 'ID_2',
 'Group',
 'Group99',
 'age_child_pre_first',
 'gender_mean',
 'chameleon_complete_stu',
 'num',
 'time',
 'a',
 'time99',
 'time99_cin',
 'N99',
 'SCARED_SUM']

In [42]:
[i for i in df_2022 if i not in df_2021]

['er_id',
 'ffq_sum_timestamp',
 'ffq_sum',
 'ffq_sum_complete',
 'c_ssrs_t_6_clin_2',
 'c_ssrs_t_7_clin_2',
 'c_ssrs_t_8_clin_2',
 'c_ssrs_t_9_clin_2',
 'c_ssrs_t_10_clin_2',
 'suicidal_behavior_last_17_clin',
 'suicidal_behavior_last_11_clin_1',
 'suicidal_behavior_last_12_clin_1',
 'suicidal_behavior_last_13_clin_1',
 'suicidal_behavior_last_18_clin',
 'suicidal_behavior_last_19_clin',
 'c_ssrs_t_17_life_clin',
 'c_ssrs_t_11_life_clin_1',
 'c_ssrs_t_12_life_clin_1',
 'c_ssrs_t_13_life_clin_1',
 'c_ssrs_t_18_life_clin',
 'c_ssrs_t_19_life_clin',
 'c_ssrs_t_6_stu_2',
 'c_ssrs_t_7_stu_2',
 'c_ssrs_t_8_stu_2',
 'c_ssrs_t_9_stu_2',
 'c_ssrs_t_10_stu_2',
 'suicidal_behavior_last_17_stu',
 'suicidal_behavior_last_11_stu_1',
 'suicidal_behavior_last_12_stu_1',
 'suicidal_behavior_last_13_stu_1',
 'suicidal_behavior_last_18_stu',
 'suicidal_behavior_last_19_stu',
 'c_ssrs_t_17_life_stu',
 'c_ssrs_t_11_life_stu_1',
 'c_ssrs_t_12_life_stu_1',
 'c_ssrs_t_13_life_stu_1',
 'c_ssrs_t_18_life_stu',

In [16]:
#diffs.to_csv('records_with_differences_between_redcups_recordings.csv', index=False)